In [ ]:
from math import gamma
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report , mean_squared_error , make_scorer , f1_score , recall_score , precision_score , fbeta_score , confusion_matrix
from matplotlib import pylab as plt
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
allData = pd.read_excel('Dataset.xlsx')

In [ ]:
InsID = 38
trainData_all = allData[allData['InstrumentID']==InsID].reset_index(drop=True)
trainData_all = trainData_all.drop(['InstrumentID'] , axis = 1)
trainData_all.fillna(0,inplace=True) 

In [ ]:
trainData = []
trainData = trainData_all.loc[0:trainData_all.shape[0]-10,['AdjustedClosePrice','AdjustedOpenPrice','AdjustedHighPrice','AdjustedLowPrice',
'EMA12','EMA21','EMA26','EMA50','EMA100','EMA200','EMA260','EMA365','MACD','MACDSignal','MACDGap',
'LNAdjustedClosePrice','LNAdjustedRealClosePrice','LNAdjustedOpenPrice','LNAdjustedHighPrice','LNAdjustedLowPrice',]]
Targets = trainData_all.loc[:trainData_all.shape[0]-10,['Target1']]
print(trainData.shape)
sc=StandardScaler()
trainData = sc.fit_transform(trainData)
indices = np.arange(trainData.shape[0])

In [ ]:
train_size = int(0.9*trainData.shape[0])

train_data = trainData[0:train_size]
train_target = Targets[0:train_size]
train_indices = indices[0:train_size]

test_data = trainData[train_size:]
test_target = Targets[train_size:]
test_indices = indices[train_size:]

In [ ]:
f_half = make_scorer(fbeta_score,beta = .2)

In [ ]:
xgbc = xgb.XGBClassifier(learning_rate  = 1 ,
 reg_alpha = 0.01 ,
 reg_lambda = 0.01 ,
 num_parallel_tree = 200 , 
 gamma  = 0.5 ,
 booster = 'gbtree' , 
 colsample_bytree = 0.4 , 
 random_state = 42 , 
 verbosity = 0 ,
 use_label_encoder = False)
Grid_params = [{'n_estimators' : [6 , 8 , 10],
          'max_depth' : [8 , 11],
                    }]

In [ ]:
CV = GridSearchCV(xgbc,
                  Grid_params,
                  cv = 5,
                  verbose = 1,
                  scoring = f_half
                  )

In [ ]:
CV.fit(train_data,train_target)

In [ ]:
print(CV.best_score_)
Mod = CV.best_estimator_
y_train = Mod.predict(train_data)
y_test = Mod.predict(test_data)


print(classification_report(train_target,y_train))
print(classification_report(test_target,y_test))